# Flights Delay Big Data Analysis

## Table of contents<a class="anchor" id="table"></a>

* [1 Working with RDD](#1)
* [1.1 Data Preparation and Loading](#1.1)
* [1.1.1 Creating SparkSession & SparkContext](#OneOneOne)
* [1.1.2 Read CSV files, Preprocessing, and final(formatted data) RDD for each file](#OneOneTwo)
* [1.1.2.1 Flights RDD](#1.1.2.1)
* [1.1.2.2 Airports RDD](#1.1.2.2)
* [1.1.3 Show RDD number of columns, and number of records](#1.1.3)
* [1.2 Dataset flights partitioning](#1.2)
* [1.2.1 Obtain the maximum arrival delay ](#1.2.1)
* [1.2.2 Obtain the minimum arrival delay ](#1.2.2)
* [1.2.3 Define hash partitioning](#1.2.3)
* [1.2.4 Display the records in each partition](#1.2.4)
* [1.3 Query RDD](#1.3)
* [1.3.1 Collect a total number of flights for each month for all flights](#1.3.1)
* [1.3.2 Collect the average delay for each month for all flights](#1.3.2)
* [2 Working with DataFrames](#2)
* [2.1 Data Preparation and Loading](#2.1)
* [2.1.1 Define DataFrames](#2.1.1)
* [2.1.2 Display the Scheme of DataFrames](#2.1.2)
* [2.1.3 Transform date-time and location column](#2.1.3)
* [2.2.1 January Flights Events with ANC airport](#2.2.1)
* [2.2.2 Average Arrival Delay From Origin to Destination](#2.2.2)
* [2.2.3 Join Query with Airports DataFrame](#2.2.3)
* [2.3 Analysis](#2.3.1)
* [2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights](#2.3.1)
* [2.3.2 Display mean arrival delay each month](#2.3.2)
* [2.3.3 Relationship between mean departure delay and mean arrival delay](#2.3.3)
* [3 RDDs vs DataFrame vs Spark SQL](#3)
* [3.1 RDD Operation](#3.1)
* [3.2 DataFrame Operation](#3.1)
* [3.3 Spark SQL Operation](#3.1)
* [3.4 Discussion](#3.1)


# 1 Working with RDD<a class="anchor" id="1"></a>
## 1.1 Data Preparation and Loading<a class="anchor" id="1.1"></a>
### 1.1.1 Create SparkSession and SparkContext<a class="anchor" id="OneOneOne"></a>
[Back to top](#table)

In [5]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Assignment 1 - Ricardo Arias (ID: 30550971)"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

### 1.1.2 Import CSV files and Make RDD for each file<a class="anchor" id="OneOneTwo"></a>
[Back to top](#table)

There are two different ways to do this, the first way could be importing the files as `CSV files` and then convert them into RDD with the function `sc.parallelize()`, and the second way that is the one we are going to use in this case is using the function `sc.textFile()` to import the CSV file as a RDD in just one step.

For this assignment we will create a function which loads the csv data into the RDD object using the function `sc.textFile()`, then we will split the data using commas (,) using the function `split()`, after we will remove the header from the RDD using `first()` and `filter`, then we will remove the **NULL** values in the dataset, changing them by 0, so it is easier to handle as number or string. Finally, we parse the RDD object into the desired format (integer, float, or string) using lists given by the user as the columns they want to convert to each fromat. All these steps are done by the function `csv_to_RDD`

In [6]:
from pyspark.sql import Row

def csv_to_RDD(path, int_cols=[], float_cols=[]): # Path of the file, list with the name of the columns that are int and float
    rdd = sc.textFile(path) # Read the CSV file and convert it into a RDD
    rdd = rdd.map(lambda line: line.split(',')) # Split data using comma (,)
    header_rdd = rdd.first() # Set the header as the first row
    rdd = rdd.filter(lambda row: row != header_rdd)   # filter out header
    rdd = rdd.map(lambda x: [0 if len(i) == 0 else i for i in x]) # Change null values with 0
    
    if (int_cols and float_cols):
        # Use map and list comprenhension to set columns as integer, float or string depending of both lists above
        rdd = rdd.map(lambda x: [int(x[i]) if header_rdd[i] in int_cols \
                                 else float(x[i]) if header_rdd[i] in float_cols else str(x[i]) for i in range(len(x))])
    
    # Convert the values from list to sql Row 
    rdd = rdd.map(lambda x: Row(**dict(zip(header_rdd, x))))
    
    return(rdd)

Now that the function is created, we proceed to import all the CSV dataset given

#### 1.1.2.1 Flights RDD <a class="anchor" id="1.1.2.1"></a>
[Back to top](#table)

Now that we have the function to import the data, we proceed to declare the columns that will be integers and floats before we  use the function to import the `flight` datasets as one

In [7]:
# Declare columns that are integer and float
int_cols = ['YEAR', 'MONTH', 'DAY','DAY_OF_WEEK', 'FLIGHT_NUMBER']
float_cols = ['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'TAXI_IN', 'TAXI_OUT']

Now we import the data using the function, and check that the information was parsed correctly

In [8]:
flights = csv_to_RDD('./flight-delays/flight*.csv', int_cols, float_cols)
flights.take(1)

[Row(YEAR=2015, MONTH=6, DAY=26, DAY_OF_WEEK=5, AIRLINE='EV', FLIGHT_NUMBER=4951, TAIL_NUMBER='N707EV', ORIGIN_AIRPORT='BHM', DESTINATION_AIRPORT='LGA', SCHEDULED_DEPARTURE='630', DEPARTURE_TIME='629', DEPARTURE_DELAY=-1.0, TAXI_OUT=13.0, WHEELS_OFF='642', SCHEDULED_TIME='155', ELAPSED_TIME=141.0, AIR_TIME=113.0, DISTANCE=866.0, WHEELS_ON='935', TAXI_IN=15.0, SCHEDULED_ARRIVAL='1005', ARRIVAL_TIME='950', ARRIVAL_DELAY=-15.0, DIVERTED='0', CANCELLED='0', CANCELLATION_REASON='0', AIR_SYSTEM_DELAY='0', SECURITY_DELAY='0', AIRLINE_DELAY='0', LATE_AIRCRAFT_DELAY='0', WEATHER_DELAY='0')]

#### 1.1.2.2 Airports RDD <a class="anchor" id="1.1.2.2"></a>
[Back to top](#table)

We do the same with the Airport dataset, but in this case we leave all the columns as `str`

In [9]:
airports = csv_to_RDD('./flight-delays/airports.csv')
airports.take(1)

[Row(IATA_CODE='ABE', AIRPORT='Lehigh Valley International Airport', CITY='Allentown', STATE='PA', COUNTRY='USA', LATITUDE='40.65236', LONGITUDE='-75.44040')]

### 1.1.3 Show RDD number of columns, and number of records <a class="anchor" id="1.1.3"></a>
[Back to top](#table)

After importing the RDD, we will display the number of columns, the total number of records, and display the number of partitions of each RDD

#### Flights

In [10]:
print("Number of Rows of Flights RDD:", flights.count(),'\nNumber of Columns of Flights RDD:', len(flights.take(1)[0]),\
      "\nNumber of Partitions of Flight RDD:",flights.getNumPartitions())

Number of Rows of Flights RDD: 582184 
Number of Columns of Flights RDD: 31 
Number of Partitions of Flight RDD: 20


#### Airports

In [11]:
print("Number of Rows of Airports RDD:", airports.count(),'\nNumber of Columns of Airports RDD:', len(airports.take(1)[0]),\
      "\nNumber of Partitions of Airports RDD:",airports.getNumPartitions())

Number of Rows of Airports RDD: 322 
Number of Columns of Airports RDD: 7 
Number of Partitions of Airports RDD: 2


## 1.2 Dataset Partitioning <a class="anchor" id="1.2"></a>

By looking at the results above we find out that the flights RDD, has 20 different partitions by default, which matches with the number of documents imported. In this particular case, by default Pyspark assume each document as one partition. 
On the other hand, the airports RDD only have 2 partitions, even though we imported 1 document. This is because the default number of partitions set by Pyspark is 2 when importing a single document. That way we can observe the two default configurations that Pyspark has.

With the RDDs already transformed we can proceed to do the parallel search

### 1.2.1 Obtain the maximum arrival delay <a class="anchor" id="1.2.1"></a>
[Back to top](#table)

Using the function of rdd `max` we can find the maximum value of a specific column. In this case we are finding the maximum ARRIVAL_DELAY in the whole dataset. That in this case is the flight 11 of the airine AS, that on 13 of July of 2015 had a delay of 1665.0 *minutes* (the units were assumed as the metadata does not mention it). 

In [12]:
flights.max(key=lambda x: x.ARRIVAL_DELAY)

Row(YEAR=2015, MONTH=9, DAY=13, DAY_OF_WEEK=7, AIRLINE='AA', FLIGHT_NUMBER=1063, TAIL_NUMBER='N3CAAA', ORIGIN_AIRPORT='SAN', DESTINATION_AIRPORT='DFW', SCHEDULED_DEPARTURE='700', DEPARTURE_TIME='1050', DEPARTURE_DELAY=1670.0, TAXI_OUT=26.0, WHEELS_OFF='1116', SCHEDULED_TIME='179', ELAPSED_TIME=174.0, AIR_TIME=142.0, DISTANCE=1171.0, WHEELS_ON='1538', TAXI_IN=6.0, SCHEDULED_ARRIVAL='1159', ARRIVAL_TIME='1544', ARRIVAL_DELAY=1665.0, DIVERTED='0', CANCELLED='0', CANCELLATION_REASON='0', AIR_SYSTEM_DELAY='0', SECURITY_DELAY='0', AIRLINE_DELAY='1665', LATE_AIRCRAFT_DELAY='0', WEATHER_DELAY='0')

### 1.2.2 Obtain the minimum arrival time <a class="anchor" id="1.2.2"></a>
[Back to top](#table)

Using the function of rdd `min` we can find the minimum value of a specific column. In this case we are finding the minimum ARRIVAL_DELAY in the whole dataset. That in this case is the same flight 11 of the airine AS, that on 21 of January of 2015 had a delay of -82.0 *minutes* (the units were assumed as the metadata does not mention it). 

In [13]:
flights.min(key=lambda x: x.ARRIVAL_DELAY)

Row(YEAR=2015, MONTH=1, DAY=21, DAY_OF_WEEK=3, AIRLINE='AS', FLIGHT_NUMBER=11, TAIL_NUMBER='N467AS', ORIGIN_AIRPORT='EWR', DESTINATION_AIRPORT='SEA', SCHEDULED_DEPARTURE='1720', DEPARTURE_TIME='1705', DEPARTURE_DELAY=-15.0, TAXI_OUT=13.0, WHEELS_OFF='1718', SCHEDULED_TIME='389', ELAPSED_TIME=322.0, AIR_TIME=305.0, DISTANCE=2402.0, WHEELS_ON='1923', TAXI_IN=4.0, SCHEDULED_ARRIVAL='2049', ARRIVAL_TIME='1927', ARRIVAL_DELAY=-82.0, DIVERTED='0', CANCELLED='0', CANCELLATION_REASON='0', AIR_SYSTEM_DELAY='0', SECURITY_DELAY='0', AIRLINE_DELAY='0', LATE_AIRCRAFT_DELAY='0', WEATHER_DELAY='0')

In [14]:
flights.max(key=lambda x: x.FLIGHT_NUMBER)

Row(YEAR=2015, MONTH=7, DAY=7, DAY_OF_WEEK=2, AIRLINE='OO', FLIGHT_NUMBER=7438, TAIL_NUMBER='N454SW', ORIGIN_AIRPORT='SLC', DESTINATION_AIRPORT='EKO', SCHEDULED_DEPARTURE='1345', DEPARTURE_TIME='1343', DEPARTURE_DELAY=-2.0, TAXI_OUT=20.0, WHEELS_OFF='1403', SCHEDULED_TIME='52', ELAPSED_TIME=58.0, AIR_TIME=33.0, DISTANCE=200.0, WHEELS_ON='1336', TAXI_IN=5.0, SCHEDULED_ARRIVAL='1337', ARRIVAL_TIME='1341', ARRIVAL_DELAY=4.0, DIVERTED='0', CANCELLED='0', CANCELLATION_REASON='0', AIR_SYSTEM_DELAY='0', SECURITY_DELAY='0', AIRLINE_DELAY='0', LATE_AIRCRAFT_DELAY='0', WEATHER_DELAY='0')

### 1.2.3 Define hash partitioning function <a class="anchor" id="1.2.3"></a>
[Back to top](#table)

Now we will define a hash function, this one will be useful to partition the data. In this case we are going to sum up all the digits of the key, and if the key is not numeric we will sum up the digits of the lenght of the string. With this hash key we guarantee an endless amount of possibilities to partition the data in as many partitions as the user wants

In [15]:
#Hash Function to implement Hash Partitioning 

def hash_function(key):
    if type(key) == str: # If the key is string
        key = len(key) # Convert the key into the lenght of the key
        
    total = 0
    for digit in str(key): # Get each digit of the key
        total += int(digit) # Sum up all the digits of the key
    return total # Return the sum

### 1.2.4 Display the records in each partition <a class="anchor" id="1.2.4"></a>
[Back to top](#table)

Then we will create the function that will allow us to observe the number of partitions and the number of records per partition in a specific RDD inputted by the user.

In [16]:
from pyspark.rdd import RDD

#A Function to print the data items in each RDD
def print_partitions(data):
    if isinstance(data, RDD):  # If the data is a RDD
        numPartitions = data.getNumPartitions() # Get the number of partitions of the RDD
        partitions = data.glom().collect() # Collect the records of each partition
    else: # If the data is not a RDD
        numPartitions = data.rdd.getNumPartitions() # Convert the data into RDD and get the number of partitions
        partitions = data.rdd.glom().collect() # Convert the data into RDD and collect the records of each partition
    
    print(f"####### NUMBER OF PARTITIONS: {numPartitions}")
    for index, partition in enumerate(partitions): # For each partition 
        # show partition if it is not empty
        if len(partition) > 0:
            print(f"Partition {index}: {len(partition)} records") # Print partition number and number of records

After we created these two functions we proceed to partition each RDD and then visualize the information of each RDD after this procedure

### Flights
The key of a RDD is the first column, in this case, the key column is year. By exploring the data we found out that all the values of this column are `2015`. Therefore, if we use this column as a key for partitioning the data, all the records would end up in the same partition, and the others will remain empty. To avoid this problem we will change the key to be `FLIGHT_NUMBER` and that way we can use the hash function correctly

In [17]:
# hash partitioning of flights RDD
flight_hash = flights.keyBy(lambda x: x.FLIGHT_NUMBER).partitionBy(4, hash_function)
print_partitions(flight_hash)

####### NUMBER OF PARTITIONS: 4
Partition 0: 145384 records
Partition 1: 147688 records
Partition 2: 145302 records
Partition 3: 143810 records


### Airports
As we did with flights RDD we explored the values of the key of airports RDD and in this case is the IATA code. It is a string so the hash function will convert it to the lenght of it, but again most of the IATA codes have only 3 digits. Therefore, if we use this column as a key for partitioning the data we will find the same error, all the records would end up in the same partition, and the others will remain empty. To avoid this problem we will change the key to be `AIRPORT` and that way we can use the hash function correctly

In [18]:
# hash partitioning of airports RDD
airports_hash = airports.keyBy(lambda x: x.AIRPORT).partitionBy(4, hash_function)
print_partitions(airports_hash)

####### NUMBER OF PARTITIONS: 4
Partition 0: 79 records
Partition 1: 85 records
Partition 2: 73 records
Partition 3: 85 records


When displaying the number of partitions of each RDD using the hash function, we see that the data is pretty balanced in both cases. Each partition has around the 25% of the data, what is equivalent to the proportion of data if it was evenlly distributed. This means that the hash function is working properly, because is preventing to overload on processor and underload some others, guarantying an optimal partitioning of the data and a correct processing of the data.

## 1.3 Query RDD  <a class="anchor" id="1.3"></a>
### 1.3.1 Collect a total number of flights for each month <a class="anchor" id="1.3.1"></a>
[Back to top](#table)

Using RDD we can not retrieve the name of the column, but in this case the key (first columns) is the number of the month. The second one is the number of flights of each month

In [19]:
flights.map(lambda x: (x.MONTH, x.FLIGHT_NUMBER)).groupByKey().mapValues(list).mapValues(lambda x: len(x)).collect()

[(1, 47136),
 (2, 42798),
 (3, 50816),
 (4, 48810),
 (5, 49691),
 (6, 50256),
 (7, 52065),
 (8, 50524),
 (9, 46733),
 (10, 48680),
 (11, 46809),
 (12, 47866)]

### 1.3.2 Collect the average delay for each month <a class="anchor" id="1.3.2"></a>
[Back to top](#table)

Following the same code we used above, in this case we are not going to count how many flights  there are per month, in this case we are going to get the average value of `ARRIVAL_DELAY` per month. The first column is the key (Number of month) and the second value is the average arrival time

In [20]:
flights.map(lambda x: (x.MONTH, x.ARRIVAL_DELAY)).groupByKey().mapValues(list)\
        .mapValues(lambda x: round(sum(x)/len(x), 2)).collect()

[(1, 5.65),
 (2, 7.72),
 (3, 4.89),
 (4, 3.14),
 (5, 4.64),
 (6, 9.53),
 (7, 6.7),
 (8, 4.65),
 (9, -0.84),
 (10, -0.54),
 (11, 0.82),
 (12, 6.04)]

# 2 Working with DataFrame <a class="anchor" id="2"></a>
## 2.1. Data Preparation and Loading <a class="anchor" id="2.1"></a>

### 2.1.1 Define dataframes and loading scheme<a class="anchor" id="2.1.1"></a>
[Back to top](#table)

To upload the data as dataframe, we will use a function that allow the user to input the path of the CSV files, and convert them into a data frame with header and a schema infered

In [21]:
def csv_to_DF(path): # Path of the file, list with the name of the columns that are int and float
     # Read the CSV file and convert it into a data frame with header, besides it has to infer the Schema
    df = spark.read.format("csv").options(header='true',inferSchema='true').load(path)
    return(df)

### 2.1.2 Display the schema of the final two dataframes<a class="anchor" id="2.1.2"></a>
[Back to top](#table)

### Flights
Now that the function is created we proceed to import the data, the specifications of the assignment says that we have to display the dataframe, but as this one has 31 columns it would be messy. Therefore, we will just display the schema of the data frame. The full dataframe would be display by using `df.show()`

In [22]:
flightsDf = csv_to_DF('./flight-delays/flight*.csv')
flightsDf.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

### Airports
We do the same with the airports dataframe

In [23]:
airportsDf = csv_to_DF('./flight-delays/airports.csv')
airportsDf.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)



It can be seen that in both cases, the code transformed the that itself. It infered the schema itself, so in this case we did not need to transform each column manually. This was achieved using 
```python
inferSchema = 'true'
```

## 2.2. Query Analysis <a class="anchor" id="2.2"></a>
### 2.2.1 January flight events with ANC airport <a class="anchor" id="2.2.1"></a>
[Back to top](#table)

### Pyspark
To do this query using Pyspark we need to import the function `col` to use filters over the columns

In [24]:
from pyspark.sql.functions import col

Now we can proceed with the query, filtering the `YEAR` before we remove it from the dataset. Then, getting the columns we are interested in, following by filtering `MONTH` and `ORIGIN_AIRPORT`

In [25]:
janFlightEventsAncDf = flightsDf.filter(col('YEAR') == 2015)\
                                .select('MONTH', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE', 'ARRIVAL_DELAY')\
                                .filter(col('MONTH') == 1)\
                                .filter(col('ORIGIN_AIRPORT') == 'ANC')

Once we have the query that we wanted, we proceed to display the table and the number of rows we extracted from it

In [26]:
print('The query has', janFlightEventsAncDf.count(), 'rows')
janFlightEventsAncDf.show()

The query has 115 rows
+-----+--------------+-------------------+--------+-------------+
|MONTH|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|ARRIVAL_DELAY|
+-----+--------------+-------------------+--------+-------------+
|    1|           ANC|                SEA|    1448|          -13|
|    1|           ANC|                SEA|    1448|           -4|
|    1|           ANC|                JNU|     571|           17|
|    1|           ANC|                CDV|     160|           20|
|    1|           ANC|                BET|     399|          -20|
|    1|           ANC|                SEA|    1448|          -15|
|    1|           ANC|                SEA|    1448|          -11|
|    1|           ANC|                ADQ|     253|          -16|
|    1|           ANC|                SEA|    1448|           17|
|    1|           ANC|                BET|     399|           -9|
|    1|           ANC|                SEA|    1448|           15|
|    1|           ANC|                FAI|     261|  

### SQL
If we perform the query using SQL we should get the same result

In [27]:
# register the original DataFrame as a temp view so that we can query it using SQL
flightsDf.createOrReplaceTempView("flightsSQL")
janFlightEventsAncDf_SQL = spark.sql('''
  SELECT MONTH, ORIGIN_AIRPORT, DESTINATION_AIRPORT, DISTANCE, ARRIVAL_DELAY FROM flightsSQL
  WHERE YEAR = 2015 
  AND MONTH = 1 
  AND ORIGIN_AIRPORT = 'ANC'
''')
print('The query has', janFlightEventsAncDf_SQL.count(), ' rows')
janFlightEventsAncDf_SQL.show()

The query has 115  rows
+-----+--------------+-------------------+--------+-------------+
|MONTH|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|ARRIVAL_DELAY|
+-----+--------------+-------------------+--------+-------------+
|    1|           ANC|                SEA|    1448|          -13|
|    1|           ANC|                SEA|    1448|           -4|
|    1|           ANC|                JNU|     571|           17|
|    1|           ANC|                CDV|     160|           20|
|    1|           ANC|                BET|     399|          -20|
|    1|           ANC|                SEA|    1448|          -15|
|    1|           ANC|                SEA|    1448|          -11|
|    1|           ANC|                ADQ|     253|          -16|
|    1|           ANC|                SEA|    1448|           17|
|    1|           ANC|                BET|     399|           -9|
|    1|           ANC|                SEA|    1448|           15|
|    1|           ANC|                FAI|     261| 

By double checking with `Pyspark` and `SQL` we discovered, that there are **115** different flights in January 2015 which their origin airport is ANC

### 2.2.2 Average Arrival Delay From Origin to Destination <a class="anchor" id="2.2.2"></a>
[Back to top](#table)

Now we import two different functions, `avg` to calculate the average value when aggregating, and `round` to get float numbers with a fixed number of decimals

In [28]:
from pyspark.sql.functions import avg
from pyspark.sql.functions import round

### Pyspark

As we did before, we are going to get this query using Pyspark

In [29]:
janFlightEventsAncAvgDf = janFlightEventsAncDf.groupBy("ORIGIN_AIRPORT", "DESTINATION_AIRPORT")\
                                                .agg(round(avg("ARRIVAL_DELAY"), 2).alias("AVG_ARRIVAL_DELAY"))\
                                                .orderBy('AVG_ARRIVAL_DELAY', ascending=False)
                                                

In [30]:
print('The query has', janFlightEventsAncAvgDf.count(), 'rows')
janFlightEventsAncAvgDf.show()

The query has 18 rows
+--------------+-------------------+-----------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AVG_ARRIVAL_DELAY|
+--------------+-------------------+-----------------+
|           ANC|                FAI|             25.0|
|           ANC|                SFO|             20.0|
|           ANC|                SCC|            16.67|
|           ANC|                LAS|              9.0|
|           ANC|                JNU|              5.0|
|           ANC|                PDX|              3.5|
|           ANC|                DEN|             3.33|
|           ANC|                PHX|              2.0|
|           ANC|                OTZ|             1.25|
|           ANC|                CDV|              1.0|
|           ANC|                ADQ|            -2.67|
|           ANC|                OME|             -3.0|
|           ANC|                BRW|            -4.33|
|           ANC|                SEA|            -6.49|
|           ANC|                BET|       

### SQL
And we should get the same result if we use SQL

In [31]:
# register the original DataFrame as a temp view so that we can query it using SQL
janFlightEventsAncDf.createOrReplaceTempView("janFlightEventsAncDfSQL")
janFlightEventsAncAvgDf_SQL = spark.sql('''
  SELECT ORIGIN_AIRPORT, DESTINATION_AIRPORT, ROUND(AVG(ARRIVAL_DELAY), 2) AS AVG_ARRIVAL_DELAY 
  FROM janFlightEventsAncDfSQL
  GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT
  ORDER BY AVG_ARRIVAL_DELAY DESC
''')
print('The query has', janFlightEventsAncAvgDf_SQL.count(), 'rows')
janFlightEventsAncAvgDf_SQL.show()

The query has 18 rows
+--------------+-------------------+-----------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AVG_ARRIVAL_DELAY|
+--------------+-------------------+-----------------+
|           ANC|                FAI|             25.0|
|           ANC|                SFO|             20.0|
|           ANC|                SCC|            16.67|
|           ANC|                LAS|              9.0|
|           ANC|                JNU|              5.0|
|           ANC|                PDX|              3.5|
|           ANC|                DEN|             3.33|
|           ANC|                PHX|              2.0|
|           ANC|                OTZ|             1.25|
|           ANC|                CDV|              1.0|
|           ANC|                ADQ|            -2.67|
|           ANC|                OME|             -3.0|
|           ANC|                BRW|            -4.33|
|           ANC|                SEA|            -6.49|
|           ANC|                BET|       

Again, by double checking with `Pyspark` and `SQL` we discovered, that there are **18** different combination of flights between ANC and other airports, besides we can see that the average delay in each route is different

### 2.2.3 Join Query with Airports DataFrame <a class="anchor" id="2.2.3"></a>
[Back to top](#table)                               

To perform the inner join we will use the `IATA_CODE` column of the airportsDf as the key column. This key could be join with `ORIGIN_AIRPORT` or `DESTINATION_AIRPORT`. That is why we are going to join them using both, so we can provide more information about the routes. In this task we will perform 2 different types of join that we could do: Sort-Merge Join and Broadcast Join

### Sort-Merge Join

#### Using ORIGIN_AIRPORT as key

In [38]:
joinedSqlDf = janFlightEventsAncAvgDf.join(airportsDf,\
                                           janFlightEventsAncAvgDf.ORIGIN_AIRPORT==airportsDf.IATA_CODE,how='inner')
print('The join has', len(joinedSqlDf.columns), 'columns')
print('The join has', joinedSqlDf.count(), 'rows')
joinedSqlDf.show()

The join has 10 columns
The join has 18 rows
+--------------+-------------------+-----------------+---------+--------------------+---------+-----+-------+--------+----------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AVG_ARRIVAL_DELAY|IATA_CODE|             AIRPORT|     CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+--------------+-------------------+-----------------+---------+--------------------+---------+-----+-------+--------+----------+
|           ANC|                BRW|            -4.33|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                ADK|            -27.0|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                OME|             -3.0|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                JNU|              5.0|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|
|           ANC|                LAS|         

#### Using DESTINATION_AIRPORT as key

In [39]:
joinedSqlDf = janFlightEventsAncAvgDf.join(airportsDf,\
                                           janFlightEventsAncAvgDf.DESTINATION_AIRPORT==airportsDf.IATA_CODE,how='inner')
print('The join has', len(joinedSqlDf.columns), 'columns')
print('The join has', joinedSqlDf.count(), 'rows')
joinedSqlDf.show()

The join has 10 columns
The join has 18 rows
+--------------+-------------------+-----------------+---------+--------------------+-------------+-----+-------+--------+----------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AVG_ARRIVAL_DELAY|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+--------------+-------------------+-----------------+---------+--------------------+-------------+-----+-------+--------+----------+
|           ANC|                BRW|            -4.33|      BRW|Wiley Post-Will R...|       Barrow|   AK|    USA|71.28545|  -156.766|
|           ANC|                ADK|            -27.0|      ADK|        Adak Airport|         Adak|   AK|    USA|51.87796|-176.64603|
|           ANC|                OME|             -3.0|      OME|        Nome Airport|         Nome|   AK|    USA| 64.5122|-165.44525|
|           ANC|                JNU|              5.0|      JNU|Juneau Internatio...|       Juneau|   AK|    USA|58.35496|-134.57628|
|           ANC| 

### Broadcast Join

#### Using ORIGIN_AIRPORT as key

In [40]:
from pyspark.sql.functions import broadcast

joinedSqlDf = airportsDf.join(broadcast(janFlightEventsAncAvgDf),\
                              janFlightEventsAncAvgDf.ORIGIN_AIRPORT==airportsDf.IATA_CODE,how='inner')
print('The join has', len(joinedSqlDf.columns), 'columns')
print('The join has', joinedSqlDf.count(), 'rows')
joinedSqlDf.show()

The join has 10 columns
The join has 18 rows
+---------+--------------------+---------+-----+-------+--------+----------+--------------+-------------------+-----------------+
|IATA_CODE|             AIRPORT|     CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AVG_ARRIVAL_DELAY|
+---------+--------------------+---------+-----+-------+--------+----------+--------------+-------------------+-----------------+
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                BET|            -9.09|
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                MSP|           -19.25|
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                SEA|            -6.49|
|      ANC|Ted Stevens Ancho...|Anchorage|   AK|    USA|61.17432|-149.99619|           ANC|                HNL|            -20.0|
|      ANC|Ted Stevens Ancho...|Anchorage|   

#### Using DESTINATION_AIRPORT as key

In [41]:
joinedSqlDf = airportsDf.join(broadcast(janFlightEventsAncAvgDf),\
                              janFlightEventsAncAvgDf.DESTINATION_AIRPORT==airportsDf.IATA_CODE,how='inner')
print('The join has', len(joinedSqlDf.columns), 'columns')
print('The join has', joinedSqlDf.count(), 'rows')
joinedSqlDf.show()

The join has 10 columns
The join has 18 rows
+---------+--------------------+-------------+-----+-------+--------+----------+--------------+-------------------+-----------------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AVG_ARRIVAL_DELAY|
+---------+--------------------+-------------+-----+-------+--------+----------+--------------+-------------------+-----------------+
|      ADK|        Adak Airport|         Adak|   AK|    USA|51.87796|-176.64603|           ANC|                ADK|            -27.0|
|      ADQ|      Kodiak Airport|       Kodiak|   AK|    USA|57.74997|-152.49386|           ANC|                ADQ|            -2.67|
|      BET|      Bethel Airport|       Bethel|   AK|    USA|60.77978|  -161.838|           ANC|                BET|            -9.09|
|      BRW|Wiley Post-Will R...|       Barrow|   AK|    USA|71.28545|  -156.766|           ANC|                BRW|            -4.33|
|      CDV|Merle 

From the joins above we can see that no matter with join strategy we use, `sort-merge` or `broadcast` join, we will get the same amount of rows and columns, the only difference would come up if we change the key. When selecting `ORIGIN_AIRPORT` as the key of joining we get 18 rows with the same values, but if we use `DESTINATION_AIRPORT` as the key we will get different information

## 2.3. Analysis <a class="anchor" id="2.3"></a>
### 2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights <a class="anchor" id="2.3.1"></a>
[Back to top](#table)

### Pyspark

In [42]:
from pyspark.sql.functions import sum as _sum
import pyspark.sql.functions as func

delayDayOfWeek = flightsDf.groupBy("DAY_OF_WEEK")\
                            .agg(round(avg("ARRIVAL_DELAY"), 2).alias("MeanArrivalDelay"),\
                                 round(_sum("ARRIVAL_DELAY"), 2).alias("TotalTimeDelay"),\
                                 func.count('ARRIVAL_DELAY').alias("NumOfFlights"))\
                            .orderBy("NumOfFlights", ascending=False)

print('The query has', len(delayDayOfWeek.columns), 'columns')
print('The query has', delayDayOfWeek.count(), 'rows')
delayDayOfWeek.show()

The query has 4 columns
The query has 7 rows
+-----------+----------------+--------------+------------+
|DAY_OF_WEEK|MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----------+----------------+--------------+------------+
|          4|            5.68|        490186|       86227|
|          5|            4.72|        401638|       85181|
|          3|            3.97|        335150|       84324|
|          1|            5.88|        494478|       84052|
|          2|            4.39|        363262|       82719|
|          7|             4.3|        343498|       79898|
|          6|            1.81|        125750|       69328|
+-----------+----------------+--------------+------------+



### SQL

In [43]:
# register the original DataFrame as a temp view so that we can query it using SQL
flightsDf.createOrReplaceTempView("flightsDfSQL")
delayDayOfWeek_SQL = spark.sql('''
  SELECT DAY_OF_WEEK, ROUND(AVG(ARRIVAL_DELAY), 2) AS MeanArrivalDelay, ROUND(SUM(ARRIVAL_DELAY), 2) AS TotalTimeDelay,
  COUNT(ARRIVAL_DELAY) AS NumOfFlights FROM flightsDfSQL
  GROUP BY DAY_OF_WEEK
  ORDER BY NumOfFlights DESC
''')

print('The query has', len(delayDayOfWeek_SQL.columns), 'columns')
print('The query has', delayDayOfWeek_SQL.count(), 'rows')
delayDayOfWeek_SQL.show()

The query has 4 columns
The query has 7 rows
+-----------+----------------+--------------+------------+
|DAY_OF_WEEK|MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----------+----------------+--------------+------------+
|          4|            5.68|        490186|       86227|
|          5|            4.72|        401638|       85181|
|          3|            3.97|        335150|       84324|
|          1|            5.88|        494478|       84052|
|          2|            4.39|        363262|       82719|
|          7|             4.3|        343498|       79898|
|          6|            1.81|        125750|       69328|
+-----------+----------------+--------------+------------+



From the query above we can identify many different things:
1. The day of the week that with the most flights is Thrusday with 86.227 during the whole year
2. The day of the week that with the least flights is Saturday with 69.328 during the whole year

Following these two pieces of information we can also observe that Thrusday is the second day of the week with more mean arrival delay, and that Saturday is the day with least delay. Besides, all days have a mean of arrival delay pretty similar, except Saturday, as well as the number of flight. This information can lead us think that exists a pretty close relationship between the number of flights per day and the mean arrival delay. 

**The more flights in a day, more arrival delay**

### 2.3.2 Display mean arrival delay each month <a class="anchor" id="2.3.2"></a>
[Back to top](#table)

### Pyspark

In [44]:
delayMonth = flightsDf.groupBy("MONTH")\
                            .agg(round(avg("ARRIVAL_DELAY"), 2).alias("MeanArrivalDelay"),\
                                 round(_sum("ARRIVAL_DELAY"), 2).alias("TotalTimeDelay"),\
                                 func.count('ARRIVAL_DELAY').alias("NumOfFlights"))\
                            .orderBy("MeanArrivalDelay", ascending=True)

print('The query has', len(delayMonth.columns), 'columns')
print('The query has', delayMonth.count(), 'rows')
delayMonth.show()

The query has 4 columns
The query has 12 rows
+-----+----------------+--------------+------------+
|MONTH|MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----+----------------+--------------+------------+
|    9|           -0.85|        -39484|       46459|
|   10|           -0.54|        -26209|       48357|
|   11|            0.83|         38412|       46203|
|    4|            3.17|        153044|       48221|
|    8|            4.71|        235063|       49866|
|    5|            4.71|        230785|       48977|
|    3|            5.01|        248454|       49580|
|    1|             5.8|        266420|       45900|
|   12|            6.16|        288883|       46909|
|    7|            6.79|        348907|       51415|
|    2|            8.12|        330513|       40684|
|    6|            9.75|        479174|       49158|
+-----+----------------+--------------+------------+



### SQL

In [45]:
# register the original DataFrame as a temp view so that we can query it using SQL
flightsDf.createOrReplaceTempView("flightsDfSQL")
delayMonth_SQL = spark.sql('''
  SELECT MONTH, ROUND(AVG(ARRIVAL_DELAY), 2) AS MeanArrivalDelay, ROUND(SUM(ARRIVAL_DELAY), 2) AS TotalTimeDelay,
  COUNT(ARRIVAL_DELAY) AS NumOfFlights FROM flightsDfSQL
  GROUP BY MONTH
  ORDER BY MeanArrivalDelay ASC
''')

print('The query has', len(delayMonth_SQL.columns), 'columns')
print('The query has', delayMonth_SQL.count(), 'rows')
delayMonth_SQL.show()

The query has 4 columns
The query has 12 rows
+-----+----------------+--------------+------------+
|MONTH|MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----+----------------+--------------+------------+
|    9|           -0.85|        -39484|       46459|
|   10|           -0.54|        -26209|       48357|
|   11|            0.83|         38412|       46203|
|    4|            3.17|        153044|       48221|
|    5|            4.71|        230785|       48977|
|    8|            4.71|        235063|       49866|
|    3|            5.01|        248454|       49580|
|    1|             5.8|        266420|       45900|
|   12|            6.16|        288883|       46909|
|    7|            6.79|        348907|       51415|
|    2|            8.12|        330513|       40684|
|    6|            9.75|        479174|       49158|
+-----+----------------+--------------+------------+



From the query above we can identify many different things:
1. The month of the year that with the least delay is September with -0.85 of mean arrival delay
2. The month of the year that with the most flights is Saturday with 9.75 of mean arrival delay

In this query we can not find a clear pattern between the number of flights and the mean arrival delay, as we did with the days of the week, which means that there are other factors different than the number of flights that can affect the mean arrival time. But unfortunately we do not have the information to find out that. 

Making some hypothesis, we could guess that the holiday season such as June, July, and December are the months that have more delay.

### 2.3.3 Relationship between mean departure delay and mean arrival delay <a class="anchor" id="2.3.3"></a>
[Back to top](#table)

### Pyspark

In [46]:
DepArrdelayMonth = flightsDf.groupBy("MONTH")\
                            .agg(round(avg("ARRIVAL_DELAY"), 2).alias("MeanArrivalDelay"),\
                                 round(avg("DEPARTURE_DELAY"), 2).alias("MeanDeptDelay"))\
                            .orderBy("MeanDeptDelay", ascending=False)

print('The query has', len(DepArrdelayMonth.columns), 'columns')
print('The query has', DepArrdelayMonth.count(), 'rows')
DepArrdelayMonth.show()

The query has 3 columns
The query has 12 rows
+-----+----------------+-------------+
|MONTH|MeanArrivalDelay|MeanDeptDelay|
+-----+----------------+-------------+
|    6|            9.75|        13.97|
|   12|            6.16|        11.82|
|    7|            6.79|        11.71|
|    2|            8.12|        11.62|
|    8|            4.71|        10.09|
|    1|             5.8|         9.75|
|    3|            5.01|         9.72|
|    5|            4.71|         9.55|
|    4|            3.17|         7.74|
|   11|            0.83|         6.63|
|   10|           -0.54|         5.24|
|    9|           -0.85|         4.73|
+-----+----------------+-------------+



### SQL

In [47]:
# register the original DataFrame as a temp view so that we can query it using SQL
flightsDf.createOrReplaceTempView("flightsDfSQL")
DepArrdelayMonth_SQL = spark.sql('''
  SELECT MONTH, ROUND(AVG(ARRIVAL_DELAY), 2) AS MeanArrivalDelay, ROUND(AVG(DEPARTURE_DELAY), 2) AS MeanDeptDelay 
  FROM flightsDfSQL
  GROUP BY MONTH
  ORDER BY MeanArrivalDelay DESC
''')

print('The query has', len(DepArrdelayMonth_SQL.columns), 'columns')
print('The query has', DepArrdelayMonth_SQL.count(), 'rows')
DepArrdelayMonth_SQL.show()

The query has 3 columns
The query has 12 rows
+-----+----------------+-------------+
|MONTH|MeanArrivalDelay|MeanDeptDelay|
+-----+----------------+-------------+
|    6|            9.75|        13.97|
|    2|            8.12|        11.62|
|    7|            6.79|        11.71|
|   12|            6.16|        11.82|
|    1|             5.8|         9.75|
|    3|            5.01|         9.72|
|    5|            4.71|         9.55|
|    8|            4.71|        10.09|
|    4|            3.17|         7.74|
|   11|            0.83|         6.63|
|   10|           -0.54|         5.24|
|    9|           -0.85|         4.73|
+-----+----------------+-------------+



Now comparing the mean arrival delay and the mean departure delay, we can see that they are closely related. The higher the departure delay, the higher the departure delay too. There is a positive correlation between these two variables, which means that if a plane has a departure delay is almost certain that it will have a departure delay. 

Even though they are closely related, the mean arrival delay is always smaller than the departure delay

# 3 RDDs vs DataFrame vs Spark SQL <a class="anchor" id="3"></a>


Implement the following queries using RDDs, DataFrames and SparkSQL separately. Log the time taken for each query in each approach using the “%%time” built-in magic command in Jupyter Notebook and discuss the performance difference of these 3 approaches.

<strong>Find the MONTH and DAY_OF_WEEK, number of flights, and average delay where TAIL_NUMBER = ‘N407AS’. Note number of flights and average delay should be aggregated separately. The average delay should be grouped by both MONTH and DAYS_OF_WEEK.</strong>

## 3.1 RDD Operation<a class="anchor" id="3.1"></a>
[Back to top](#table)

In [62]:
%%time
%%timeit

rdd_numFlights = flights.filter(lambda x: x.TAIL_NUMBER == 'N407AS')\
                        .map(lambda x: ((x.MONTH, x.DAY_OF_WEEK), x.ARRIVAL_DELAY))\
                        .groupByKey().mapValues(list)\
                        .mapValues(lambda x: (len(x), sum(x)/len(x)))

22.8 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
CPU times: user 1.11 s, sys: 73.3 ms, total: 1.18 s
Wall time: 2.04 s


In [63]:
rdd_numFlights.take(15)

[((11, 7), (3, -4.0)),
 ((3, 3), (1, 3.0)),
 ((4, 6), (1, -20.0)),
 ((12, 2), (2, -11.5)),
 ((5, 5), (1, 6.0)),
 ((6, 3), (3, -10.666666666666666)),
 ((7, 4), (2, -4.0)),
 ((8, 1), (2, -13.0)),
 ((9, 2), (1, -10.0)),
 ((6, 2), (1, 35.0)),
 ((9, 3), (5, -14.6)),
 ((7, 5), (1, -4.0)),
 ((8, 7), (2, 60.5)),
 ((10, 1), (2, 15.5)),
 ((9, 4), (3, -10.666666666666666))]

## 3.2 DataFrame Operation<a class="anchor" id="3.2"></a>
[Back to top](#table)

In [71]:
%%time
%%timeit


df_numFlights = flightsDf.filter(flightsDf.TAIL_NUMBER == 'N407AS')\
                            .groupBy("MONTH", "DAY_OF_WEEK")\
                            .agg(func.count('ARRIVAL_DELAY').alias("NumOfFlights"),\
                                 round(avg("ARRIVAL_DELAY"), 2).alias("MeanArrivalDelay"))

25.8 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
CPU times: user 384 ms, sys: 73 ms, total: 457 ms
Wall time: 2.51 s


In [72]:
df_numFlights.show()

+-----+-----------+------------+----------------+
|MONTH|DAY_OF_WEEK|NumOfFlights|MeanArrivalDelay|
+-----+-----------+------------+----------------+
|    6|          1|           4|             7.0|
|    3|          1|           1|            29.0|
|    7|          4|           2|            -4.0|
|    2|          2|           2|            -9.5|
|    9|          4|           3|          -10.67|
|   12|          7|           2|            -1.0|
|    8|          3|           1|            -4.0|
|    4|          7|           1|           -22.0|
|    2|          3|           2|           -11.5|
|    7|          1|           1|            -1.0|
|   12|          2|           2|           -11.5|
|    1|          2|           2|            17.5|
|   11|          1|           1|            35.0|
|    9|          1|           2|           -15.5|
|    5|          7|           3|           -7.67|
|    5|          6|           2|            -3.0|
|   12|          1|           1|            -1.0|


## 3.3 Spark SQL OPERATION<a class="anchor" id="3.3"></a>
[Back to top](#table)

In [74]:
%%time
%%timeit

# register the original DataFrame as a temp view so that we can query it using SQL
flightsDf.createOrReplaceTempView("flightsDfSQL")
df_numFlights_SQL = spark.sql('''
  SELECT MONTH, DAY_OF_WEEK, COUNT(ARRIVAL_DELAY) AS NumOfFlights, ROUND(AVG(ARRIVAL_DELAY), 2) AS MeanArrivalDelay, 
  ROUND(AVG(DEPARTURE_DELAY), 2) AS MeanDeptDelay
  FROM flightsDfSQL
  WHERE TAIL_NUMBER == 'N407AS'
  GROUP BY MONTH, DAY_OF_WEEK
  ORDER BY MeanDeptDelay DESC
''')

43.3 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
CPU times: user 52.1 ms, sys: 1.33 ms, total: 53.4 ms
Wall time: 3.59 s


In [75]:
df_numFlights_SQL.show()

+-----+-----------+------------+----------------+-------------+
|MONTH|DAY_OF_WEEK|NumOfFlights|MeanArrivalDelay|MeanDeptDelay|
+-----+-----------+------------+----------------+-------------+
|    8|          7|           2|            60.5|         66.5|
|   11|          1|           1|            35.0|         57.0|
|    3|          1|           1|            29.0|         40.0|
|    6|          2|           1|            35.0|         33.0|
|    3|          3|           1|             3.0|         28.0|
|    5|          5|           1|             6.0|         17.0|
|    7|          7|           4|            19.0|         15.8|
|    1|          2|           2|            17.5|         12.5|
|   10|          1|           2|            15.5|         12.5|
|    7|          5|           1|            -4.0|          9.0|
|    1|          6|           3|            4.33|         8.67|
|    9|          2|           1|           -10.0|          8.0|
|    5|          2|           5|        

## 3.4 Discussion<a class="anchor" id="3.4"></a>
[Back to top](#table)

Doing the same query using 3 different strategies `RDD`, `Pyspark Dataframe` and `SQL Dataframe` we find out pretty interesting things. The first one is the time that each query takes to perform, `RDD` is the quickest one, only taking **2.04s** to complete which means *~22.8ms* per loop. The sencond one is the `Pyspark Dataframe`, taking **2.51s** to complete or *~25.8ms* per loop. And the slowest one is the `SQL Dataframe` taking **3.59s** to complete or *~43.3ms* per loop. 

In the following table we can see the comparison between strategies:

|Strategy|Total Time (s)|Time per Loop (ms)|% of delay vs. RDD|
|:-:|:-:|:-:|:-:|
|RDD|2.04|~22.8|-|
|Pyspark|2.51|~25.8|23.3%|
|SQL|3.59|~43.3|89.9%|

From the table above we can understand some things, as it was already mentioned, the RDD strategy is the most efficient one if we only take in count the time of processing, but we also habe to be aware that when working with RDD the output is a list, and not a table, that is why the query does not have headers, which makes it not easy to read. Therefore, if we measure the efficiency by readability the `RDD` is not the best. In this case `Pyspark` would be the best, as the result is a dataset easy to read. While the only advantage that the `SQL dataframe` query could present is the readability of the code.

In order to identify the best strategy to do the query, we must understand the purpuse of the query, and then we will understand which one is the best.